In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import pandas as pd

df_preds = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/STA130/test_predictions.csv')
df_preds

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/STA130/test_predictions.csv'

In [ ]:

df_preds = pd.read_csv('test_predictions.csv')
df_preds

In [ ]:
df_indicators = pd.read_csv('country_indicators.csv')
df_indicators

In [ ]:
df = df_preds.merge(df_indicators, left_on='iso3', right_on='iso3', how='inner')
df

In [ ]:
# Okay, so we first need to get the Probability Prediciton "Error"
import numpy as np
df['error_transformer'] = np.abs(df.y_true_transformer-df.y_pred_proba_transformer)
df['error_ffnn'] = np.abs(df.y_true_ffnn-df.y_pred_proba_ffnn)
df['error_xgboost'] = np.abs(df.y_true_xgboost-df.y_pred_proba_xgboost)

In [ ]:
# But, then, what I wanted to do first was look at those "Progress Indicator" variables that kept
# getting mentioned; but, specifically, the actual indicator variable versions of these...
# This is code Evan Wheeler gave me and it does the trick to make the indicator variables I want

# make df of column names and their dtypes
df_cols = pd.DataFrame(df.dtypes, columns=('coldtype',)).reset_index().rename(columns={'index': 'colname'})
df_cols['coldtype'] = df_cols['coldtype'].astype('string')
# adjusting a potentially useful variable that might be considered label
df['fsi_rank'] = df['fsi_rank'].astype('string').str.replace(r'\D', '', regex=True).replace('', pd.NA)
# get list of numeric variables
num_vars = df_cols.query("coldtype=='float64'")['colname'].values
# these could be useful; but, I'm ignoring them for now to keep the demonstration simpler...

import itertools

# Indicator variables are often called "one hot" encodings
def one_hot(df, cols):
    """ One-hot encode given `cols` and add as new columns
        to `df`

        Returns tuple of `df` with new columns and list of
        new column names.
    """
    new_cols = list()
    new_col_names = list()
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each)
        new_cols.append(dummies)
        new_col_names.append(dummies.columns.values)

    df = pd.concat([df]+new_cols, axis=1)
    new_col_names = list(itertools.chain.from_iterable(new_col_names))
    return df, new_col_names

# categorical variables we will turn into indicator ("one hot") variables
cat_vars = ['hdr_region', 'wbi_income_group', 'wbi_lending_category']
cont_vars = ['fsi_x1:_external_intervention', 'sowc_education__learning_literacy-rate-2014-2022_youth-15-24-years-literacy-rate_male', 'fsi_p1:_state_legitimacy', 'fsi_p2:_public_services', 'fsi_p3:_human_rights', 'fsi_c1:_security_apparatus', 'fsi_s1_demographic_pressures']
# Prof. Schwartz'z Note:
# I'm not sure if there are other categorical variables in this data that could also be transformed
# These are just the ones Evan initially used as examples as he described the objectives of the project
# get one hot encodings
df_oh, oh_cols = one_hot(df, cat_vars)
df_oh = df_oh.drop(columns=cat_vars)
cont_col = []


df_oh[['error_transformer','error_ffnn','error_xgboost'] + oh_cols]



In [ ]:
# For the analysis demonstration below I'm not going to use the following indicator variables:
# `'fsi_category_Stable', 'hdr_hdicode_Low', 'wbi_income_group_High income',
# 'wbi_lending_category_IBRD', 'wbi_other_(emu_or_hipc)_HIPC'` and `'hdr_region_SSA'`
# This makes these categories the "baseline" categories for each of these variables.

# I initially thought I was choosing these because I thought the names suggested
# they might not be a conflict region; or, I just kinda randomly picked one
# because I couldn't tell what they meant...
# I should have probably first tried to figure out what they meant...

# But, anyway, for now, this means my analysis is just using the following indicator variables:
reduced_oh_cols = \
[
 'hdr_region_AS',
 'hdr_region_EAP',
 'hdr_region_ECA',
 'hdr_region_LAC',
 'hdr_region_SA',
 'wbi_income_group_Low income',
 'wbi_income_group_Lower middle income',
 'wbi_income_group_Upper middle income',
 'wbi_lending_category_Blend',
 'wbi_lending_category_IDA',
 'fsi_x1:_external_intervention',
 'fsi_s1:_demographic_pressures',
 'fsi_p1:_state_legitimacy',
 'fsi_p2:_public_services',
 'fsi_p3:_human_rights',
 'fsi_c1:_security_apparatus',
 'sowc_demographics__population-thousands-2021_total',
 'sowc_demographics__life-expectancy-at-birth-years_2000-0',
 'sowc_demographics__life-expectancy-at-birth-years_1970',
 'sowc_demographics__population-thousands-2021_under-18',
 'fsi_e3:_human_flight_and_brain_drain',
 'sowc_demographics__annual-number-of-births-thousands-2021_2020-2030-a',
 'sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a',
 'sowc_migration__international-migrant-stock-2020_total-thousands'


]
# And so for now use them and take a look at them below

In [ ]:
df_oh['model_transformer'] = df_oh['error_transformer'].astype(str)*0+"transformer"
df_oh['model_ffnn'] = df_oh['error_ffnn'].astype(str)*0+"ffnn"
df_oh['model_xgboost'] = df_oh['error_xgboost'].astype(str)*0+"xgboost"


## Correlation heatmap
>The correlation heatmap gives a visusal representation of the correlation between each indicator. In this case, a dark color of blue or red suggests a high correlation, either proportional or inversely proportional. If two covariates have high correlation, this introduces multicollinearity in the model, where two covariates predict in similar ways. This weakens the predicting power of the regression, hence it is to our interest to remove one of them.

In [ ]:
# Do indicator values correlate with model performance values?
import matplotlib.pyplot as plt
import seaborn as sns

# A good function from Evan Wheeler I've edited just a little bit
def corr_heatmap(df):
    # plot correlation heatmap based on code from:
    # https://medium.com/@nikolh92/helpful-visualisations-for-linear-regression-646a5648ad9d
    sns.set(style="white")
    corr = df.corr()
    mask = np.zeros_like(corr, dtype=bool)
    #mask[np.triu_indices_from(mask)] = True
    fig, ax = plt.subplots(figsize=(20, 16))
    cmap = sns.diverging_palette(10, 220, as_cmap=True)
    return sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0, square=True,
                       linewidths=.5, annot=False, cbar_kws={"shrink": .5})

corr_heatmap(df_oh[['error_transformer','error_ffnn','error_xgboost'] + reduced_oh_cols])
_ = plt.axhline(y=3, c='k'); plt.axvline(x=3, c='k')

## Removing indicators with high correlation


In [ ]:
reduced_oh_cols.remove("sowc_demographics__population-thousands-2021_total")
reduced_oh_cols.remove("sowc_demographics__annual-number-of-births-thousands-2021_2020-2030-a")
reduced_oh_cols.remove("fsi_p1:_state_legitimacy")
reduced_oh_cols.remove("sowc_demographics__life-expectancy-at-birth-years_1970")
reduced_oh_cols.remove("sowc_demographics__life-expectancy-at-birth-years_2000-0")

In [ ]:
# Do indicator values correlate with model performance values?
import matplotlib.pyplot as plt
import seaborn as sns

# A good function from Evan Wheeler I've edited just a little bit
def corr_heatmap(df):
    # plot correlation heatmap based on code from:
    # https://medium.com/@nikolh92/helpful-visualisations-for-linear-regression-646a5648ad9d
    sns.set(style="white")
    corr = df.corr()
    mask = np.zeros_like(corr, dtype=bool)
    #mask[np.triu_indices_from(mask)] = True
    fig, ax = plt.subplots(figsize=(20, 16))
    cmap = sns.diverging_palette(10, 220, as_cmap=True)
    return sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0, square=True,
                       linewidths=.5, annot=False, cbar_kws={"shrink": .5})

corr_heatmap(df_oh[['error_transformer','error_ffnn','error_xgboost'] + reduced_oh_cols])
_ = plt.axhline(y=3, c='k'); plt.axvline(x=3, c='k')

In [ ]:
# This will be used to make the (additional) indicator variables of which model a prediction corresponds to
df_oh['model_transformer'] = df_oh['error_transformer'].astype(str)*0+"transformer"
df_oh['model_ffnn'] = df_oh['error_ffnn'].astype(str)*0+"ffnn"
df_oh['model_xgboost'] = df_oh['error_xgboost'].astype(str)*0+"xgboost"

# The amount of error might change depending on if the prediction is positive or negative
# so this indicates if it was a postive or negative prediction
df_oh['prediction_transformer'] = df.y_true_transformer.astype(int) # actual outcome whether its conflict or no conflict?
df_oh['prediction_ffnn'] = df.y_pred_ffnn.astype(int)
df_oh['prediction_xgboost'] = df.y_true_xgboost.astype(int)

# This "stacks" all the predictions together on top of each other so they can all be analyzed together:
# (make sure you understand what this is actually doing and why!)
design_matrix = \
pd.concat([df_oh[['error_transformer', 'model_transformer', 'prediction_transformer']+reduced_oh_cols].rename(columns={'error_transformer':'error','model_transformer':'model','prediction_transformer':'predicts1'}),
           df_oh[['error_ffnn', 'model_ffnn', 'prediction_ffnn']+reduced_oh_cols].rename(columns={'error_ffnn':'error','model_ffnn':'model','prediction_ffnn':'predicts1'}),
           df_oh[['error_xgboost', 'model_xgboost', 'prediction_xgboost']+reduced_oh_cols].rename(columns={'error_xgboost':'error','model_xgboost':'model','prediction_xgboost':'predicts1'})],
          ignore_index=True)

# 0. design_matrix[reduced_oh_cols] effects at "baseline" (`ffnn` predicts 0)

# 1. design_matrix[reduced_oh_cols]*predicts1 offset changes to "baseline" when prediction is 1
design_matrix.predicts1 # is the intercept offset
for x in reduced_oh_cols:
    design_matrix[x+' X predicts1'] = design_matrix[x]*design_matrix['predicts1']

# 2. design_matrix[reduced_oh_cols]*`transformer`/`xgboost` additional offset changes to "baseline"
# when prediction is made by `transformer`/`xgboost` for any prediction (0 or 1)
design_matrix['transformer'] = (design_matrix['model']=="transformer").astype(int) # intercept offset
design_matrix['xgboost'] = (design_matrix['model']=="xgboost").astype(int) # intercept offset
for x in reduced_oh_cols:
    design_matrix[x+' X transformer'] = design_matrix[x]*design_matrix['transformer']
    design_matrix[x+' X xgboost'] = design_matrix[x]*design_matrix['xgboost']

# 3. design_matrix[reduced_oh_cols]*`transformer_predicts1`/`xgboost_predicts1`
# additional offset changes to "baseline" for non `ffnn` 1 predictions
design_matrix['transformer X predicts1'] = design_matrix['transformer']*design_matrix['predicts1']
design_matrix['xgboost X predicts1'] = design_matrix['xgboost']*design_matrix['predicts1']
for x in reduced_oh_cols:
    design_matrix[x+' X transformer X predicts1'] = design_matrix[x]*design_matrix['transformer X predicts1']
    design_matrix[x+' X xgboost X predicts1'] = design_matrix[x]*design_matrix['xgboost X predicts1']

# This is to address the "DataFrame is highly fragmented" warning that's being flagged below
design_matrix = design_matrix.copy()
y = design_matrix['error']
del design_matrix['error']
del design_matrix['model']
design_matrix#.columns

In [ ]:
import statsmodels.api as sm

model_0_variables = design_matrix.columns.tolist()
model_0 = sm.OLS(y, sm.add_constant(design_matrix[model_0_variables]))
model_0.fit().summary()

# Assuming model_0.fit().summary() returns a Summary object
#summary = model_0.fit().summary()

# Extract the p-values and coefficients into a DataFrame
#summary_df = pd.DataFrame(summary.tables[1].data[1:], columns=summary.tables[1].data[0])

# Convert p-values to numeric (they may be strings)
#summary_df['P>|t|'] = pd.to_numeric(summary_df['P>|t|'])

# Sort by p-value in descending order
#sorted_summary = summary_df.sort_values(by='P>|t|', ascending=False)

# Display or use sorted_summary as needed
# print(sorted_summary)

In [ ]:
design_matrix.shape
#doing so we see that there are interactions that do not occur many times

In [ ]:
design_matrix.loc[:,design_matrix.sum()>20].shape

##  We can see that there are 27 interactions that occured less than 20 times. Because interactions with low occurance rates makes it difficult to estimate the error prediction rate, as they did not occur many  We'll restrict ourselves to only examing categories that happen at least 20 or more times.  


In [ ]:
design_matrix.loc[:,design_matrix.sum() > 20].shape #for interpretability

In [ ]:
model_1_variables = design_matrix.loc[:,design_matrix.sum()>20].columns.tolist()
model_1 = sm.OLS(y, sm.add_constant(design_matrix[model_1_variables]))
model_1.fit().summary()

We will be doing backwards selection. Backward selection is a variable selection method used in the context of regression analysis. The goal of backward selection is to iteratively remove the least significant predictors from a model until a satisfactory or optimal model is achieved.

GPT



## We can remove interactions with a p value greater than 0.10 first. This is because we want to do an initial filtering to examine what the model is having removed the interactions with a p value that have no evidence against the null hypothesis.

In [ ]:
model_2_variables = model_1_variables.copy()

# Manual "Backwards Selection"
# - I removed the variables below, bottom to top
# - To see my steps comment the removals, then uncomment one at a time, from bottom to top
#   re-running the cell each time#model_2_variables.remove("sowc_demographics__population-thousands-2021_under-18")#
model_2_variables.remove("hdr_region_EAP X transformer")
model_2_variables.remove("hdr_region_ECA X transformer")
model_2_variables.remove("wbi_income_group_Upper middle income X transformer")
model_2_variables.remove("fsi_x1:_external_intervention X transformer")#
model_2_variables.remove("sowc_demographics__population-thousands-2021_under-18")
model_2_variables.remove("sowc_demographics__population-thousands-2021_under-18 X xgboost X predicts1")
model_2_variables.remove("sowc_demographics__population-thousands-2021_under-18 X predicts1")#
model_2_variables.remove("fsi_x1:_external_intervention")#
model_2_variables.remove("fsi_c1:_security_apparatus")#
model_2_variables.remove("sowc_demographics__population-thousands-2021_under-18 X xgboost")#
model_2_variables.remove("fsi_p3:_human_rights X xgboost")#
model_2_variables.remove("wbi_income_group_Low income")#
model_2_variables.remove("fsi_p2:_public_services X xgboost X predicts1")#
model_2_variables.remove("wbi_income_group_Lower middle income X transformer")#
model_2_variables.remove("fsi_e3:_human_flight_and_brain_drain")#
model_2_variables.remove("fsi_p3:_human_rights")#
model_2_variables.remove("hdr_region_AS")#
model_2_variables.remove("wbi_income_group_Low income X predicts1")#
model_2_variables.remove("wbi_lending_category_IDA X transformer")#
model_2_variables.remove("fsi_s1:_demographic_pressures X transformer")#
model_2_variables.remove("fsi_s1:_demographic_pressures X xgboost X predicts1")#
model_2_variables.remove("fsi_p3:_human_rights X transformer X predicts1")#
model_2_variables.remove("wbi_lending_category_IDA X xgboost")#
model_2_variables.remove("wbi_lending_category_Blend X predicts1")#
model_2_variables.remove("sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a")#
model_2_variables.remove("hdr_region_ECA")#
model_2_variables.remove("hdr_region_SA")#
model_2_variables.remove("xgboost")#
model_2_variables.remove("hdr_region_ECA X xgboost")#
model_2_variables.remove("fsi_p3:_human_rights X predicts1")#
model_2_variables.remove("wbi_income_group_Lower middle income X xgboost X predicts1")#
model_2_variables.remove("fsi_x1:_external_intervention X transformer X predicts1")#
model_2_variables.remove("sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a X transformer X predicts1")#
model_2_variables.remove("sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a X xgboost X predicts1")#
model_2_variables.remove("fsi_p3:_human_rights X xgboost X predicts1")#
model_2_variables.remove("xgboost X predicts1")#
model_2_variables.remove("fsi_x1:_external_intervention X xgboost")#
model_2_variables.remove("hdr_region_LAC X transformer")#
model_2_variables.remove("fsi_x1:_external_intervention X predicts1")#
model_2_variables.remove("fsi_s1:_demographic_pressures X predicts1")#
model_2_variables.remove("fsi_p2:_public_services X xgboost")#
model_2_variables.remove("sowc_migration__international-migrant-stock-2020_total-thousands X transformer")#
model_2_variables.remove("sowc_migration__international-migrant-stock-2020_total-thousands X xgboost")#
model_2_variables.remove("sowc_migration__international-migrant-stock-2020_total-thousands X transformer X predicts1")#
model_2_variables.remove("wbi_lending_category_IDA X transformer X predicts1")#
model_2_variables.remove("sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a X predicts1")#
model_2_variables.remove("fsi_e3:_human_flight_and_brain_drain X transformer")#
model_2_variables.remove("wbi_income_group_Low income X xgboost")#
model_2_variables.remove("wbi_income_group_Low income X transformer")#
model_2_variables.remove("transformer")#
model_2_variables.remove("fsi_p2:_public_services X predicts1")#
model_2_variables.remove("wbi_income_group_Upper middle income X predicts1")#
model_2_variables.remove("fsi_e3:_human_flight_and_brain_drain X xgboost X predicts1")#
model_2_variables.remove("fsi_c1:_security_apparatus X predicts1")#
model_2_variables.remove("fsi_p2:_public_services")#
model_2_variables.remove("wbi_income_group_Upper middle income")#
model_2_variables.remove("hdr_region_EAP")#
model_2_variables.remove("wbi_lending_category_IDA")#
model_2_variables.remove("wbi_lending_category_IDA X predicts1")#
model_2_variables.remove("sowc_migration__international-migrant-stock-2020_total-thousands X predicts1")#
model_2_variables.remove("sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a X transformer")#
model_2_variables.remove("fsi_x1:_external_intervention X xgboost X predicts1")#
model_2_variables.remove("sowc_migration__international-migrant-stock-2020_total-thousands X xgboost X predicts1")#
model_2_variables.remove("wbi_lending_category_Blend")#
model_2_variables.remove("hdr_region_AS X xgboost")#
model_2_variables.remove("hdr_region_LAC") #0.955
model_2_variables.remove("fsi_s1:_demographic_pressures X xgboost")#0.979
model_2_variables.remove("predicts1") #0.991


model_2 = sm.OLS(y, sm.add_constant(design_matrix[model_2_variables]))
model_2.fit().summary()

Now all of our interactions are below the p value of 0.1 , indicating there is weak evidence against hypothesis or stronger
notes:
- cond number is really high suggesting multicolinearity
- r squared is 0.5, not bad
- removed the larger p values first (this was kind of rough we might need to go back i just sped through.) removed the ones above 0.9, then 0.5, and the 0.2- 0.1. This is important as removing interactions with very high p value influences the p values of the rest
-

In [ ]:
model_3_variables = model_2_variables.copy()

model_3_variables.remove("sowc_demographics__population-thousands-2021_under-18 X transformer X predicts1")
model_3_variables.remove("hdr_region_EAP X xgboost")
model_3_variables.remove("fsi_p3:_human_rights X transformer")
model_3_variables.remove("sowc_demographics__net-migration-rate-per-1000-population-2021_old-age-dependency-ratio_2020-2030-a X xgboost")
model_3_variables.remove("sowc_migration__international-migrant-stock-2020_total-thousands")
model_3_variables.remove("fsi_s1:_demographic_pressures X transformer X predicts1")
model_3_variables.remove("fsi_p2:_public_services X transformer X predicts1")
model_3_variables.remove("fsi_e3:_human_flight_and_brain_drain X transformer X predicts1")
model_3_variables.remove("fsi_e3:_human_flight_and_brain_drain X predicts1")
model_3_variables.remove("wbi_income_group_Lower middle income")
model_3_variables.remove("wbi_income_group_Lower middle income X predicts1")
model_3_variables.remove("sowc_demographics__population-thousands-2021_under-18 X transformer")
model_3_variables.remove("hdr_region_AS X transformer")
model_3_variables.remove("wbi_income_group_Upper middle income X xgboost")

model_3 = sm.OLS(y, sm.add_constant(design_matrix[model_3_variables]))
model_3.fit().summary()

We have removed all interactions with a p value greater than 0.
This is good! however, the cond number is still too high at 15k. r^2 is around 0.4 which is not bad

In [ ]:
model_4_variables = model_3_variables.copy()


model_4_variables.remove("fsi_c1:_security_apparatus X transformer X predicts1")
model_4_variables.remove("transformer X predicts1")

model_4 = sm.OLS(y, sm.add_constant(design_matrix[model_4_variables]))
model_4.fit().summary()

predicts1 is 1 when the model predicts 1

- Selectively removed indicators which resulted in multicolinearity. Multicolinearity is not a problem anymore, and the model should be good to go and interpretable.
- r^2 proportion of test statistic explained. it is the explanatory power of the model and is around 0.3 is adequate :D
- p values are all 0, we can stop here for now.

In [ ]:
model_4.fit().params

# Final regression equations for the three conflict prediction models:

## $\hat y_{ffnn} = 0.2031 + 0.0345({demographic\_pressures})$

## $\hat y_{xgboost} = 0.2031 + 0.0345({demographic\_pressures}) + 0.0936 I_{latin\_america}(hdr\_region) + 0.0672I_{[lower\_middle]}(wbi\_income\_group) + 0.0437({security\_apparatus}) -0.0299({security\_apparatus) \times(predicts1})$  

## $\hat y_{transformer} = 0.2031 + 0.0345({demographic\_pressures}) -0.030631 ({public\_services}) + 0.037268(security\_apparatus)$


In [ ]:
import plotly.express as px
fig = px.scatter(df_oh, )

# recommendations:
> limitations within model, we did not consider many columns with empty values so most of the dataset

## Modeling the residuals
> We can see that the residuals follow a normal distribution by assumption. We are taking the y minus the y hat, the predicted value of our model. We can see that the normality peaks at the around 0, which is roughly what the mean is of the distrubtion. This follows the assumption of multilinear regression.

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
_ = plt.hist(y-model_4.fit().predict())# y - yhat this is residuals
plt.xlabel('Residuals')
plt.ylabel('Count')
plt.title("Barchart of the distribution of residuals")

## We are plotting the $\hat y$ against the $y - \hat y$,

In [ ]:
_ = plt.plot(model_4.fit().predict(), y-model_4.fit().predict(), '.')
#x axis is y hat + noise, y is residuals
#variance of the residuals appear to be constant, hence follows the assumption of the model
#clusters?
#bottom are going to -4, top going to 0.6 so variance end up being similar either way
plt.xlabel('Predictions')
plt.ylabel('Residuals')
plt.title("Scatterplot of the predictions graphed against the residuals")

# Training and testing:

In [ ]:
np.random.seed(130)
train_size = 800
data_indices = np.random.choice(design_matrix.index, size=design_matrix.shape[0], replace=False)
train_indices = data_indices[train_size:]
test_indices = data_indices[:train_size]

model_1_train_test_fit = sm.OLS(y[train_indices], sm.add_constant(design_matrix.iloc[train_indices][model_1_variables])).fit()
model_1_train_RMSE = ((y[train_indices] - model_1_train_test_fit.predict())**2).mean()**.5
model_1_test_RMSE = ((y[test_indices] -
                      model_1_train_test_fit.predict(sm.add_constant(design_matrix.iloc[test_indices][model_1_variables]))
                     )**2).mean()**.5

model_2_train_test_fit = sm.OLS(y[train_indices], sm.add_constant(design_matrix.iloc[train_indices][model_2_variables])).fit()
model_2_train_RMSE = ((y[train_indices] - model_2_train_test_fit.predict())**2).mean()**.5
model_2_test_RMSE = ((y[test_indices] -
                      model_2_train_test_fit.predict(sm.add_constant(design_matrix.iloc[test_indices][model_2_variables]))
                     )**2).mean()**.5

model_3_train_test_fit = sm.OLS(y[train_indices], sm.add_constant(design_matrix.iloc[train_indices][model_3_variables])).fit()
model_3_train_RMSE = ((y[train_indices] - model_3_train_test_fit.predict())**2).mean()**.5
model_3_test_RMSE = ((y[test_indices] -
                      model_3_train_test_fit.predict(sm.add_constant(design_matrix.iloc[test_indices][model_3_variables]))
                     )**2).mean()**.5

model_4_train_test_fit = sm.OLS(y[train_indices], sm.add_constant(design_matrix.iloc[train_indices][model_4_variables])).fit()
model_4_train_RMSE = ((y[train_indices] - model_4_train_test_fit.predict())**2).mean()**.5
model_4_test_RMSE = ((y[test_indices] -
                      model_4_train_test_fit.predict(sm.add_constant(design_matrix.iloc[test_indices][model_4_variables]))
                     )**2).mean()**.5

import plotly.express as px
px.bar(pd.DataFrame({'RMSE': [model_1_train_RMSE, model_2_train_RMSE, model_3_train_RMSE, model_4_train_RMSE] +
                             [model_1_test_RMSE, model_2_test_RMSE, model_3_test_RMSE, model_4_test_RMSE],
                     'Score': ['Training']*4+['Testing']*4,
                     'Model': [1,2,3,4]+[1,2,3,4]}),
       y='RMSE', x='Model', color='Score', barmode='group', title = "Bar chart of model performances on training and testing data")


## Notes:

> RMSE, or root mean squared error is a metric to assess the performance of a predictive model in a linear regression. The higher the RMSE the farther the predictions of the model is from the actual values.

> The RMSE in model 4 is actually higher in comparison to model 3 and the RMSE in model 2 was actually the lowest out of the 4 models. While the increase is RMSE is very minimal, it still suggests that we may have removed useful interactions during our backwards selection process. Interestingly enough, this was not due to overfitting, as the RMSE in the training data went up as well.

> Relatively similar RMSE for both training and testing datasets in models 2, 3, and 4 suggests the model is fairly capable of predicting the error rate. However, the condition numbers for model 2 and 3 are is too high, with high multicolinearity present in the model. Hence, it reduces the interpretability of the covariates in the model, since multiple covariates would be influencing the error prediction rate by the same magnitude.



In [4]:
model_3_train_test_fit.summary().tables[1]

NameError: name 'model_3_train_test_fit' is not defined

In [ ]:
model_4_train_test_fit.summary().tables[1]

We see that the p values for some indicators have changed from being 0 to a greater number.

In [ ]:
reps = 100
model_3_train_RMSEs = np.array([0.0]*reps)
model_4_train_RMSEs = np.array([0.0]*reps)
model_3_test_RMSEs = np.array([0.0]*reps)
model_4_test_RMSEs = np.array([0.0]*reps)
for i in range(reps):
    data_indices = np.random.choice(design_matrix.index, size=design_matrix.shape[0], replace=False)
    train_indices = data_indices[train_size:]
    test_indices = data_indices[:train_size]

    model_3_train_test_fit = sm.OLS(y[train_indices], sm.add_constant(design_matrix.iloc[train_indices][model_3_variables])).fit()
    model_3_train_RMSE = ((y[train_indices] - model_3_train_test_fit.predict())**2).mean()**.5
    model_3_test_RMSE = ((y[test_indices] -
                          model_3_train_test_fit.predict(sm.add_constant(design_matrix.iloc[test_indices][model_3_variables]))
                         )**2).mean()**.5
    model_3_train_RMSEs[i] = model_3_train_RMSE
    model_3_test_RMSEs[i] = model_3_test_RMSE

    model_4_train_test_fit = sm.OLS(y[train_indices], sm.add_constant(design_matrix.iloc[train_indices][model_4_variables])).fit()
    model_4_train_RMSE = ((y[train_indices] - model_4_train_test_fit.predict())**2).mean()**.5
    model_4_test_RMSE = ((y[test_indices] -
                          model_4_train_test_fit.predict(sm.add_constant(design_matrix.iloc[test_indices][model_4_variables]))
                         )**2).mean()**.5
    model_4_train_RMSEs[i] = model_4_train_RMSE
    model_4_test_RMSEs[i] = model_4_test_RMSE

In [ ]:
px.scatter(pd.DataFrame({'Incresed Test RMSE': (model_3_test_RMSEs-model_4_train_RMSEs).tolist()+
                                               (model_4_test_RMSEs-model_4_train_RMSEs).tolist(),
                         'Training RMSE': model_3_train_RMSEs.tolist()+model_4_train_RMSEs.tolist(),
                         'Model': ["Model 3"]*reps+["Model 4"]*reps}),
           x='Training RMSE', y='Incresed Test RMSE', color='Model',
           marginal_x="box", marginal_y="violin",title="Performance of model 3 and 4 on testing data")

Its better for the models to predict conflict and then there is no positive than predict no conflict and there is actually conflict. Hence, the prediction error might be more positive.


If the coefficient is positive, it increases the error prediction rate on the countries with that attribute. If the coefficinet is negative, then it decreases the prediction error rate, hence the model does well on predicting the error rate.

## Improvements
- To further improve our linear regression, we can use forward selection to reintroduce interactions that may have had significance in the error prediction rate of the model.  
- While we were careful during the backwards selection process, we still may have removed interactions that had significant relationship to the error prediction rate. We could trace back and see if any labels are worthwhile introducing back to the regression model.
- Because we were only able to perform backward selection on a limited amount of indicators, there are plentiful indicators that we did not manage to include in our backward selection process. To expand upon our final model, we could implement forward selection to find the interactions in columns that we did not consider to further increase the statistical power of our regression model.